In [1]:
!pip install --quiet pypdf openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 17.0 MB/s eta 0:00:00


In [2]:
!pip install --quiet langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.1 MB/s eta 0:00:00


In [5]:
!pip install --quiet langchain-community langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [6]:
!pip install --quiet --no-deps langchain-community langchain-text-splitters

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import openai

print("All imports successful")

All imports successful


In [8]:

# Reading the file and splitting them
pdf_reader = PyPDFLoader("/content/Sales.pdf")
documents = pdf_reader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200,)
chunks = text_splitter.split_documents(documents)

In [13]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 69.1 MB/s eta 0:00:00


In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# Create embeddings using OpenAI
embeddings = OpenAIEmbeddings(api_key="your_api_key")
# Create vector store
db = FAISS.from_documents(documents=chunks, embedding=embeddings)

In [18]:
!pip install -U langchain-xai

In [ ]:

# Calling the model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key="your_api_key"
)

In [41]:
llm.invoke('Who is Roger Federer?')

AIMessage(content='Roger Federer is a Swiss professional tennis player, widely regarded as one of the greatest tennis players of all time. Born on August 8, 1981, in Basel, Switzerland, he turned professional in 1998. Federer is known for his elegant playing style, exceptional shot-making ability, and sportsmanship on and off the court.\n\nThroughout his career, Federer has won a total of 20 Grand Slam singles titles, including eight Wimbledon titles, six Australian Open titles, five US Open titles, and one French Open title. He has held the world No. 1 ranking in the ATP rankings for a record total of 310 weeks, including a record 237 consecutive weeks.\n\nIn addition to his success on the court, Federer is also known for his philanthropic efforts, particularly through the Roger Federer Foundation, which focuses on education and sports for children in Africa and Switzerland. He has a significant global fan base and is considered a cultural icon in the world of sports. Federer announce

In [47]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_core.runnables import RunnableParallel, RunnableLambda

def format_docs(docs):
  if not docs:
    return "No documents found"
  return "\n\n".join(doc.page_content for doc in docs)

def retrieve_and_format(question):
  docs = db.as_retriever().invoke(question)
  return { "context": format_docs(docs), "question": question }

qa = ( RunnableLambda(retrieve_and_format) | qa_prompt | llm | StrOutputParser() )

In [49]:
response = qa.invoke("What is sales?")
print(response)


Sales is the process of selling products or services to customers, which involves various activities aimed at generating revenue for a business. It encompasses the strategies and efforts made by sales teams to meet specific targets and objectives, ultimately contributing to the company's overall growth and success. Sales is not just about hitting revenue targets; it also involves aligning with corporate goals, understanding market positioning, and enhancing customer perception. A well-defined sales strategy is crucial for ensuring that sales activities are data-backed and aligned with the long-term vision of the organization.


In [50]:
response2 = qa.invoke("Who is Bryan Caldwell?")
print(response2)

Bryan Caldwell is a 42-year-old surgeon based in Chicago, IL, working in the healthcare industry. He holds a doctorate and is motivated by the need for high-quality surgical equipment to ensure patient welfare. His goals include earning a reputation as the "go-to" surgeon in the Chicago area, and he faces the pain point of equipment hindering his performance. In his personal life, he enjoys spending time with family, socializing with friends at a yachting club, and watching the Bears on Sundays.
